In [1]:
import tequila as tq
import numpy as np
from tequila.hamiltonian import QubitHamiltonian, paulis
from tequila.grouping.binary_rep import BinaryHamiltonian

The following examples shows how to partition a given Hamiltonian into commuting parts and how to find the unitary transformation needed to transform the commuting terms into qubit-wise commuting form that is easy to measure. 

The Hamiltonian is simply 
$$ H = \sigma_z(0)\sigma_z(1) + \sigma_y(0)\sigma_y(1) + \sigma_x(0)\sigma_x(1) + \sigma_x(0)$$
where $\sigma_z(0)\sigma_z(1)$, $\sigma_y(0)\sigma_y(1)$ does not commute with $\sigma_x(0)$, so two separate measurements are needed.

In [2]:
H = paulis.Z(0) * paulis.Z(1) + paulis.Y(0) * paulis.Y(1) + \
    paulis.X(0) * paulis.X(1) + paulis.X(0) 

Here we use the binary representation of the Hamiltonian for partitioning. The method commuting_groups gets back a list of BinaryHamiltonian whose terms are mutually commuting. 

Call to_qubit_hamiltonian to visualize.

In [3]:
binary_H = BinaryHamiltonian.init_from_qubit_hamiltonian(H)
commuting_parts = binary_H.commuting_groups()

In [4]:
print(len(commuting_parts)) # Number of measurements needed
print(commuting_parts[0].to_qubit_hamiltonian())
print(commuting_parts[1].to_qubit_hamiltonian())

2
+0.0000i+1.0000X(0)X(1)+1.0000X(0)
+0.0000i+1.0000Z(0)Z(1)+1.0000Y(0)Y(1)


The second group of terms $H_2$ are not currently qubit-wise commuting and cannot be directly measured on current hardware. They require further unitary transformation $U$ to become qubit-wise commuting. The following code identifies two bases (list of BinaryPauliString) that encodes the unitary transformation as
$$ U = \prod_i \frac{1}{2} (\text{old_basis}[i] + \text{new_basis}[i])$$
such that $UH_2U$ is qubit-wise commuting.

In [5]:
qubit_wise_parts, old_basis, new_basis = commuting_parts[1].get_qubit_wise()

In [6]:
def display_basis(basis):
    for term in basis:
        print(QubitHamiltonian.init_from_paulistring(term.to_pauli_strings()))
print('Old Basis')
display_basis(old_basis)
print('\nNew Basis')
display_basis(new_basis)

Old Basis
+1.0000Y(0)Y(1)
+1.0000X(0)X(1)

New Basis
+1.0000X(0)
+1.0000Y(1)


The transfromed term $UH_2U$ is qubit-wise commuting. 

In [7]:
print(qubit_wise_parts.to_qubit_hamiltonian())

+0.0000i-1.0000X(0)Y(1)+1.0000X(0)


In [8]:
old_basis

Trying to construct the circuit for the unitary transformation to implement the measurement scheme

In [159]:
def to_qubit_operator(bps):
    return tq.QubitHamiltonian.init_from_paulistring(bps.to_pauli_strings())

UM = tq.gates.QCircuit()
UM_op = tq.paulis.I()
for i in range(len(old_basis)):
    sigma = to_qubit_operator(old_basis[i])
    tau = to_qubit_operator(new_basis[i])
    print("anti-comm  = ", sigma*tau + tau*sigma) # seems to be fine
    UM_op *= 1.0/np.sqrt(2.0)*(sigma + tau) # think its sqrt(2) and not 2 (see above)
    V0=tq.gates.ExpPauli(angle=-tq.numpy.pi/2, paulistring=sigma.paulistrings[0]) # gates are defined as exp(-angle/2 * generator) 
    V1=tq.gates.ExpPauli(angle=-tq.numpy.pi/2, paulistring=tau.paulistrings[0])
    V2=tq.gates.ExpPauli(angle=-tq.numpy.pi/2, paulistring=sigma.paulistrings[0])
    UM += V0 + V1 + V2


anti-comm  =  
anti-comm  =  


In [160]:
UM

circuit: 
Exp-Pauli(target=(0, 1), control=(), parameter=-1.5707963267948966, paulistring=+1.0000Y(0)Y(1))
Rx(target=(0,), parameter=-1.5707963267948966)
Exp-Pauli(target=(0, 1), control=(), parameter=-1.5707963267948966, paulistring=+1.0000Y(0)Y(1))
Exp-Pauli(target=(0, 1), control=(), parameter=-1.5707963267948966, paulistring=+1.0000X(0)X(1))
Ry(target=(1,), parameter=-1.5707963267948966)
Exp-Pauli(target=(0, 1), control=(), parameter=-1.5707963267948966, paulistring=+1.0000X(0)X(1))

In [161]:
newH = (qubit_wise_parts.to_qubit_hamiltonian()).simplify()

In [162]:
# try measurement scheme
U = tq.gates.Ry(angle="a", target=0) + tq.gates.Ry(angle="b", target=1)
E1 = tq.ExpectationValue(H=H, U=U)
E2 = tq.ExpectationValue(H=newH, U=U+UM)
variables = {"a":1.0 , "b": 1.0}
print(tq.simulate(E1, variables=variables))
print(tq.simulate(E2, variables=variables))


1.8414709568023682
0.2919265925884247


In [163]:
# test if unitary (1.0/sqrt(2.0) factor seems to be fine)
UM_op*UM_op

+1.0000+0.0000iX(0)Z(1)+0.0000iZ(0)Y(1)+0.0000iY(0)X(1)

In [164]:
# test if transformed operator is the same as newH
# seems not to be the case
(UM_op*newH*UM_op).simplify()

+1.0000Z(0)Z(1)+1.0000Y(0)Y(1)

In [165]:
newH

-1.0000X(0)Y(1)+1.0000X(0)

In [166]:
# test if circuit is correct
# seems to be the case
wfn = tq.simulate(U, variables=variables)
wfn2 = wfn.apply_qubitoperator(UM_op)
wfn3 = tq.simulate(U+UM, variables=variables)

In [167]:
wfn2

+0.3692e^(-0.6569πi)|00> +0.3692e^(+0.1569πi)|10> +0.6030e^(+0.0506πi)|01> +0.6030e^(+0.4494πi)|11> 

In [168]:
wfn3

+0.3692e^(+0.3431πi)|00> +0.3692e^(-0.8431πi)|10> +0.6030e^(-0.9494πi)|01> +0.6030e^(-0.5506πi)|11> 

In [169]:
np.abs(wfn2.inner(wfn3))**2

0.9999999999999993